<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Colab" data-toc-modified-id="Colab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Colab</a></span></li><li><span><a href="#Load-the-libraries" data-toc-modified-id="Load-the-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the libraries</a></span></li><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the data</a></span></li><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train Test Split</a></span></li><li><span><a href="#Modelling:-pyGAM" data-toc-modified-id="Modelling:-pyGAM-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modelling: pyGAM</a></span></li><li><span><a href="#Model-evaluation" data-toc-modified-id="Model-evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Model evaluation</a></span></li></ul></div>

# Load the libraries

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import os,sys,time
import sklearn
import scipy
import matplotlib.pyplot as plt
sns.set()

import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_tweedie_deviance
from sklearn.metrics import auc

SEED = 100
pd.set_option('max_columns',100)
pd.set_option('plotting.backend','matplotlib') # matplotlib, bokeh, altair, plotly
%load_ext watermark
%watermark -iv

sklearn    0.22.1
scikitplot 0.3.7
numpy      1.19.1
pandas     1.1.1
json       2.0.9
seaborn    0.11.0
scipy      1.4.1
joblib     0.16.0



# Load the data

In [16]:
%%bash
pwd

/Users/poudel/github/Data_Science/a01_Modules/pyGAM/example


In [26]:
# ifile = '../data/processed/clean_data.csv.zip'
ifile = "https://github.com/bhishanpdl/Datasets/blob/master/Projects/French_Motor_Claims/processed/clean_data.csv.zip?raw=true"
df = pd.read_csv(ifile, compression='zip')
print(df.shape)
df.head(2).append(df.tail(2))

(100000, 15)


,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
1,0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
99998,0,0.90,C,7,9,44,50,B1,Regular,191,R24,0.0,0.0,0.0,0.0
99999,0,0.90,E,4,12,53,50,B1,Regular,4116,R24,0.0,0.0,0.0,0.0


In [31]:
# ifile = "../data/processed/X.npz"
ifile = os.path.expanduser("~/github/Project_French_Motor_Claims/data/processed/X.npz")

X = scipy.sparse.load_npz(ifile)
X.shape, type(X)

((100000, 71), scipy.sparse.csr.csr_matrix)

In [32]:
# ifile = '../data/processed/X.csv.zip'
# df_X = pd.read_csv(ifile, compression='zip')
# print(df_X.shape)
# df_X.head(2).append(df_X.tail(2))

# columns
"""
x0_B1	x0_B10	x0_B11	x0_B12	x0_B13	x0_B14	x0_B2	x0_B3	x0_B4	x0_B5
x0_B6	x1_4	x1_5	x1_6	x1_7	x1_8	x1_9	x1_10	x1_11	x1_12
x1_13	x1_14	x1_15	x2_Diesel	x2_Regular	x3_R11	x3_R21	x3_R22	x3_R23
x3_R24	x3_R25	x3_R26	x3_R31	x3_R41	x3_R42	x3_R43	x3_R52	x3_R53	x3_R54
x3_R72	x3_R73	x3_R74	x3_R82	x3_R83	x3_R91	x3_R93	x3_R94	x4_A	x4_B
x4_C	x4_D	x4_E	x4_F	VehAge_0	VehAge_1	VehAge_2	VehAge_3
VehAge_4	VehAge_5	VehAge_6	VehAge_7	DrivAge_0	DrivAge_1	DrivAge_2	
DrivAge_3	DrivAge_4	DrivAge_5	DrivAge_6	DrivAge_7	Density	BonusMalus

X only have transformed version of these columns
cols_ohe_before = ["VehBrand", "VehPower", "VehGas", "Region", "Area"]
cols_kbin_before = ["VehAge", "DrivAge"]
cols_log_scale = ["Density"]
cols_pass =  ["BonusMalus"]
""";

In [33]:
df.head(2)

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
0,0,0.10,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0
1,0,0.77,D,5,0,55,50,B12,Regular,1217,R82,0.0,0.0,0.0,0.0


In [34]:
np.array(X[0].todense())[0][-5:] # last elements of first row

array([ 0.        ,  1.        ,  0.        ,  0.69864446, 50.        ])

# Train Test Split

In [35]:
from sklearn.model_selection import train_test_split

In [44]:
df_train, df_test, X_train, X_test = train_test_split(df, X.todense(), random_state=SEED)

df_train.shape, df_test.shape, X_train.shape, X_test.shape

((75000, 15), (25000, 15), (75000, 71), (25000, 71))

# Modelling: pyGAM
Ref: https://pygam.readthedocs.io/en/latest/notebooks/tour_of_pygam.html
```
Method            link        distribution
----------------------------------------------------------
LinearGAM         identity    normal distribution
LogisticGAM logit link        binomial distribution
PoissonGAM        log         Poisson distribution
GammaGAM          log         gamma distribution
InvGauss          log         inv_gauss distribution
```

LinearGAM
$\mathbb{E}[y \mid X]=\beta_{0}+f_{1}\left(X_{1}\right)+f_{2}\left(X_{2}, X 3\right)+\cdots+f_{M}\left(X_{N}\right)$

**Parameters**  
```
Terms
l() linear terms
s() spline terms
f() factor terms
te() tensor products
intercept
```

**Callbacks**  
Callbacks are performed during each optimization iteration. It’s also easy to write your own.
```
deviance - model deviance
diffs - differences of coefficient norm
accuracy - model accuracy for LogisticGAM
coef - coefficient logging
```

In [45]:
import pygam

In [46]:
df_train.head(2)

,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,ClaimAmount,PurePremium,Frequency,AvgClaimAmount
17853,0,0.35,D,6,7,25,147,B1,Diesel,745,R82,0.0,0.0,0.0,0.0
55890,0,0.61,E,5,1,24,72,B3,Diesel,3673,R25,0.0,0.0,0.0,0.0


In [53]:
y_train = df_train["AvgClaimAmount"].values
y_test = df_test["AvgClaimAmount"].values

In [49]:
%%time
gam = pygam.LinearGAM(n_splines=10).gridsearch(X_train, y_train)
gam.summary()

100% (11 of 11) |########################| Elapsed Time: 0:03:51 Time:  0:03:51


LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                     67.4214
Link Function:                     IdentityLink Log Likelihood:                               -1148621.477
Number of Samples:                        75000 AIC:                                          2297379.7967
                                                AICc:                                         2297379.9234
                                                GCV:                                          1789834.1192
                                                Scale:                                        1786938.1978
                                                Pseudo R-Squared:                                   0.0019
Feature Function                  Lam

/Users/poudel/opt/miniconda3/envs/gam/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  


# Model evaluation

In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_tweedie_deviance
from sklearn.metrics import auc

In [50]:
tr_preds = gam.predict(X_train)
tx_preds = gam.predict(X_test)

In [54]:
tr_mae = mean_absolute_error(y_train,tr_preds)
tx_mae = mean_absolute_error(y_test,tx_preds)

tr_mse = mean_squared_error(y_train, tr_preds)
tx_mse = mean_squared_error(y_test,tx_preds)

df_eval_gam = pd.DataFrame(
{'train': [tr_mae, tr_mse],
'test': [tx_mae, tx_mse]})

df_eval_gam.index = ['mean_absolute_error','mean_squared_error']
df_eval_gam

,train,test
mean_absolute_error,1.686438e+02,1.655408e+02
mean_squared_error,1.785332e+06,1.647533e+06
